# Area Assumptions

This notebook creates graphs for each group and examines if they differ from random graphs.

The first part of the script imports the data and creates graphs.

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import scona as scn
from scona.visualisations import plot_network_measures, plot_degree_dist, plot_rich_club
import matplotlib.pyplot as plt
import nilearn.plotting as plot
import seaborn as sns
sns.set_style('dark')

This code creates the enviornmental variable for where the data is stored. Create a .env file in the directory with the file path to data.

In [2]:
from decouple import config

data = config('data')

In [3]:
lh_area = pd.read_csv(f'{data}/lh_area.dat',sep='\t').drop(['lh_WhiteSurfArea_area',
       'BrainSegVolNotVent', 'eTIV'],axis=1).rename(columns={'lh.aparc.area':'G-Number'})

rh_area =  pd.read_csv(f'{data}/rh_area.dat',sep='\t').drop(['rh_WhiteSurfArea_area',
       'BrainSegVolNotVent', 'eTIV','rh.aparc.area'],axis=1)

group = pd.read_csv(f'{data}/cortical_measures.csv').iloc[0:,2]

area = pd.concat([lh_area, rh_area, group],axis=1)

names = list(area.columns.drop(['G-Number','age_adjusted_group']))

centroids = pd.read_csv(f'{data}/atlas.csv') #This atlas was obtained from braingraph R package

centroids = centroids[['x.mni',	'y.mni', 'z.mni']].to_numpy()

group = area.groupby('age_adjusted_group')
aan = group.get_group('AAN').reset_index(drop=True)
hc = group.get_group('HC').reset_index(drop=True)
wr = group.get_group('WR').reset_index(drop=True)

In [4]:
aan_residuals_df = scn.create_residuals_df(aan.iloc[:,1:69], names)
aan_corr_matrix = scn.create_corrmat(aan_residuals_df, method='pearson')
aan_graph = scn.BrainNetwork(network=aan_corr_matrix, parcellation=names, centroids=centroids)
aan_graph_threshold = aan_graph.threshold(10)
print(aan_graph_threshold)

BrainNetwork with 68 nodes and 228 edges


In [ ]:
wr_residuals_df = scn.create_residuals_df(wr.iloc[:,1:69], names)
wr_corr_matrix = scn.create_corrmat(wr_residuals_df, method='pearson')
wr_graph = scn.BrainNetwork(network=wr_corr_matrix, parcellation=names, centroids=centroids)
wr_graph_threshold = wr_graph.threshold(10)

In [ ]:
hc_residuals_df = scn.create_residuals_df(hc.iloc[:,1:69], names)
hc_corr_matrix = scn.create_corrmat(hc_residuals_df, method='pearson')
hc_graph = scn.BrainNetwork(network=hc_corr_matrix, parcellation=names, centroids=centroids)
hc_graph_threshold = hc_graph.threshold(10)

## Creation of Random Graphs.

This simulates random graphs to check that the actual graph differ from random graphs

In [5]:
brain_bundle_aan = scn.GraphBundle([aan_graph_threshold], ['AAN_graph_thresholded'])
brain_bundle_aan.create_random_graphs('AAN_graph_thresholded', 100)

        Creating 100 random graphs - may take a little while


In [ ]:
brain_bundle_wr = scn.GraphBundle([wr_graph_threshold], ['WR_graph_thresholded'])
brain_bundle_wr.create_random_graphs('WR_graph_thresholded', 1000)

In [ ]:
brain_bundle_hc = scn.GraphBundle([hc_graph_threshold], ['HC_graph_thresholded'])
brain_bundle_hc.create_random_graphs('HC_graph_thresholded', 1000)

## Calculating nodal and global measures

In [ ]:
aan_graph_threshold.calculate_nodal_measures()
nodal_df_aan = aan_graph_threshold.report_nodal_measures()

In [ ]:
wr_graph_threshold.calculate_nodal_measures()
nodal_df_wr = wr_graph_threshold.report_nodal_measures()

In [ ]:
hc_graph_threshold.calculate_nodal_measures()
nodal_df_hc = hc_graph_threshold.report_nodal_measures()

In [ ]:
random_graph_measures_aan = brain_bundle_aan.report_global_measures()
random_graph_measures_aan.head()

In [ ]:
random_graph_measures_wr = brain_bundle_wr.report_global_measures()
random_graph_measures_wr.head()

In [ ]:
random_graph_measures_hc = brain_bundle_hc.report_global_measures()
random_graph_measures_hc.head()

## Plotting degree distribution

In [ ]:
plot_degree_dist(aan_graph_threshold, color=["purple", "black"])

In [ ]:
plot_degree_dist(wr_graph_threshold, color=["navy", "black"])

In [ ]:
plot_degree_dist(hc_graph_threshold, color=["darkorange", "black"])

## Plotting global Measures.

In [ ]:
plot_network_measures(brain_bundle_aan, original_network='AAN_graph_thresholded', color=['purple','slategrey'], ci=95)

In [ ]:
plot_network_measures(brain_bundle_wr, original_network='WR_graph_thresholded', color=['navy','slategrey'], ci=95)

In [ ]:
plot_network_measures(brain_bundle_hc, original_network='HC_graph_thresholded', color=['darkorange','slategrey'], ci=95)

## Plotting rich club

In [ ]:
plot_rich_club(brain_bundle_aan, color=['purple','slategrey'], original_network='AAN_graph_thresholded')

In [ ]:
plot_rich_club(brain_bundle_wr, color=['navy','slategrey'], original_network='WR_graph_thresholded')

In [ ]:
plot_rich_club(brain_bundle_hc, color=['darkorange','slategrey'], original_network='HC_graph_thresholded')